## Show the SFS dynamics varying the params

Simulated data generated by `simulations.sh parameters_varying_eta` and the others, from `parameters.ipynb`.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path
from typing import List, Dict, Set

from hscpy import mitchell, realisation, parameters
from hscpy.figures import PlotOptions, ToCellFrequency
from hscpy.figures import sfs as sfs_fig

from futils import parse_version, snapshot

BIGLABELS = False
FIGSIZE = [5, 3] if BIGLABELS else [6.4, 4.8]  # default matplotlib
LATEST = True
SAVEFIG = True
EXTENSION = ".svg"
PATH2HSC = Path("~").expanduser() / "hsc"
PLOT_OPTIONS = PlotOptions(figsize=FIGSIZE, extension=EXTENSION, save=SAVEFIG)

In [ ]:
%%bash -s "$PATH2HSC" --out version
$1/target/release/hsc  --version

In [ ]:
if LATEST:
    VERSION = parse_version(version)
else:
    VERSION = "sfs/v3.0.6"
PATH2SAVE = Path(f"./{VERSION}")

print("Running hsc with version:", VERSION)

In [ ]:
PATH2SIMS = f"dynamics/{VERSION}"

In [ ]:
donors = mitchell.donors()
donors

In [ ]:
def loading_sfs_varying_param(param: str) -> Dict:
    sfs_sims = dict()
    if param == "eta":
        p = "etas"
    elif param == "mu":
        p = "mus"
    elif param == "tau":
        p = "taus"
    else:
        raise ValueError("wrong value of param")
        
    for donor in donors[donors.name.isin({"KX001", "KX008"})].itertuples():
        sfs_sims[donor.name] = realisation.load_all_sfs_by_age(
           Path(f"{PATH2SIMS}/{p}/{donor.cells}cells/sfs")
        )[donor.age]
    return sfs_sims

In [ ]:
def plot_fig_params(sfs_idx: List[Set], donors, filename: str):
    for sfs_s in sfs_idx:
        fig, ax = plt.subplots(1, 1, layout="constrained", figsize=PLOT_OPTIONS.figsize)
        markers, linestyles, colors = (".", "x"), ("-", "--"), ("#737373", "#bdbdbd")
        for color, ls, marker, (name, s) in zip(colors, linestyles, markers, sfs_sims.items()):
            subset = [sfs for sfs in s if sfs.parameters.idx in sfs_s]
            cells, age = donors[donors.name == name].cells.squeeze(), donors[donors.name == name].age.squeeze()
            sfs_fig.plot_sfs_with_avg(
                ax, 
                subset, 
                subset[1],
                age,
                cells,
                marker,
                PLOT_OPTIONS,
                color=color,
                ls=ls,
            )
        ax.legend(fontsize="small")
        params = subset[1].parameters.into_dict()
        s, std, mu, tau = params["s"], params["std"], params["mu"], params["tau"]
        ax.text(
            x=0.5, 
            y=0.7, 
            s=f"$\eta={{{s/tau:.2f}}}, \sigma={{{std/tau:.2f}}}$",
            transform=ax.transAxes
        )
        ax.text(
            x=0.5, 
            y=0.6, 
            s=f"$\mu={{{mu}}}, τ={{{tau}}}$",
            transform=ax.transAxes
        )
        if PLOT_OPTIONS.save:
            fig.savefig(filename)
        plt.show()

In [ ]:
sfs_sims = loading_sfs_varying_param("eta")
sfs_small_eta = {ele.parameters.idx for x in sfs_sims.values() for ele in x if ele.parameters.s == 0.05}
sfs_medium_eta = {ele.parameters.idx for x in sfs_sims.values() for ele in x if ele.parameters.s > 0.05 and ele.parameters.s < 0.14}
sfs_high_eta = {ele.parameters.idx for x in sfs_sims.values() for ele in x if ele.parameters.s > 0.14}
plot_fig_params([sfs_small_eta, sfs_medium_eta, sfs_high_eta], donors, f"./sfs_dynamics_varying_eta{PLOT_OPTIONS.extension}")

In [ ]:
sfs_sims = loading_sfs_varying_param("mu")
sfs_small_mu = {ele.parameters.idx for x in sfs_sims.values() for ele in x if ele.parameters.mu == 0.5}
sfs_medium_mu = {ele.parameters.idx for x in sfs_sims.values() for ele in x if ele.parameters.mu == 3}
sfs_high_mu = {ele.parameters.idx for x in sfs_sims.values() for ele in x if ele.parameters.mu == 10}
plot_fig_params([sfs_small_mu, sfs_medium_mu, sfs_high_mu], donors, f"./sfs_dynamics_varying_mu{PLOT_OPTIONS.extension}")

In [ ]:
sfs_sims = loading_sfs_varying_param("tau")
sfs_small_tau = {ele.parameters.idx for x in sfs_sims.values() for ele in x if ele.parameters.tau == 0.3}
sfs_medium_tau = {ele.parameters.idx for x in sfs_sims.values() for ele in x if ele.parameters.tau == 1}
sfs_high_tau = {ele.parameters.idx for x in sfs_sims.values() for ele in x if ele.parameters.tau == 5}
plot_fig_params([sfs_small_tau, sfs_medium_tau, sfs_high_tau], donors, f"./sfs_dynamics_varying_tau{PLOT_OPTIONS.extension}")